In [1]:
!pip install hill_climbing scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 103.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import cuml

# Setting Matplotlib defaults
plt.style.use('seaborn-v0_8')
plt.rc('figure', figsize=(8,5), dpi=150)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=15, titlepad=10)
plt.rc('animation', html='html5')
plt.tight_layout()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', 500)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/xgb-cat-ensemble-model/oof_preds.parquet
/kaggle/input/xgb-cat-ensemble-model/__results__.html
/kaggle/input/xgb-cat-ensemble-model/submission.csv
/kaggle/input/xgb-cat-ensemble-model/__notebook__.ipynb
/kaggle/input/xgb-cat-ensemble-model/test_preds.parquet
/kaggle/input/xgb-cat-ensemble-model/__output__.json
/kaggle/input/xgb-cat-ensemble-model/custom.css
/kaggle/input/xgb-cat-ensemble-model/catboost_info/test_error.tsv
/kaggle/input/xgb-cat-ensemble-model/catboost_info/learn_error.tsv
/kaggle/input/xgb-cat-ensemble-model/catboost_info/catboost_training.json
/kaggle/input/xgb-cat-ensemble-model/catboost_info/time_left.tsv
/kaggle/input/xgb-cat-ensemble-model/catboost_info/learn/events.out.tfevents
/kaggle/input/xgb-cat-ensemble-model/catboost_info/test/events.out.tfevents
/kaggle/input/calories-burnt-prediction/calories.csv
/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.

<Figure size 1200x750 with 0 Axes>

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv', index_col='id')
org = pd.read_csv('/kaggle/input/calories-burnt-prediction/calories.csv', index_col='User_ID')
org = org.rename(columns={'Gender': 'Sex'})

In [4]:
train = pd.concat([train, org], ignore_index=True)

In [5]:
strt_cols = list(test.columns)
print(train.drop_duplicates(subset=strt_cols + ['Calories']).shape, train.shape)

(762107, 8) (765000, 8)


In [6]:
X = train.copy()
y = X.pop('Calories')
y = np.log1p(y)

X_test = test.copy()


In [7]:
oof_preds = pd.read_parquet("/kaggle/input/xgb-cat-ensemble-model/oof_preds.parquet")
test_preds = pd.read_parquet("/kaggle/input/xgb-cat-ensemble-model/test_preds.parquet")

In [8]:
from hill_climbing import Climber, ClimberCV
from sklearn.metrics import root_mean_squared_error

In [9]:
climber = Climber(
    objective='minimize',
    eval_metric=root_mean_squared_error,
    allow_negative_weights=True,
    precision=0.01,
    score_decimal_places=6,
    random_state=100,
    n_jobs=1,
    use_gpu=True
).fit(oof_preds, y)


Configuration

   Metric:                       root_mean_squared_error
   Objective:                    minimize
   Precision:                    0.01
   Allow negative weights:       True
   Starting model:               best
   Number of parallel jobs:      1
   Number of models:             2
   Using GPU:                    False


Models

   XGB   0.059152 █ (best)
   CAT   0.059260 ████████████████████████████████████████


Running Hill Climbing

   Iter   Model     Weight        Score      Improvement         Time
   ────────────────────────────────────────────────────────────────
      0   XGB       1.00     0.059152                -            -
      1   CAT       0.46     0.058825         0.000326         5.12


Results

   Number of models in ensemble:      2
   Overall improvement:               +0.000326 (+0.55%)
   Total time:                        9.22 seconds
   Average iteration time:            4.56 seconds
   Final score:                       0.058825


In [10]:
final_preds = climber.predict(test_preds)

In [11]:
sub = pd.read_csv('/kaggle/input/playground-series-s5e5/sample_submission.csv')
sub['Calories'] = final_preds
sub.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
print(sub.head(5))

Your submission was successfully saved!
       id    Calories
0  750000   27.166972
1  750001  107.659760
2  750002   87.598418
3  750003  125.520047
4  750004   75.717026
